In [4]:
from IPython.display import clear_output

!pip install -U accelerate
!pip install -U transformers
!pip install datasets
!pip install evaluate

clear_output()

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import datasets
from datasets import load_dataset
import evaluate

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List
import string

import pandas as pd
import random
import os

import numpy as np
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
sns.set(palette='Paired')
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import re
import time
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv(r'D:\data science\T1_hol\train.csv')

In [3]:
df

,index,review,sentiment
0,521911,"Есть много причин, по которым 'Война и мир' Ль...",1
1,776224,"Напишите 5 предложений. 1. ""Война и мир"" — это...",2
2,141250,[Практикуйте «Другие люди»] Отзыв о «Войне и ...,1
3,678689,Стремитесь к точному и объективному представле...,2
4,175974,"В книге ""Война и мир"" Льва Толстого разворачив...",2
...,...,...,...
23995,312953,"Вот и все, что нужно сделать: Во-первых, объяс...",1
23996,804602,"Обязательно укажите, что именно в тексте/сюжет...",0
23997,616585,"Кто, из героев вас заинтересовал и почему? Как...",1
23998,477513,"Жду историй о том, как книга изменила ваше вос...",1


In [4]:
df = df.drop('index',axis = 1)

In [5]:
df2 = pd.DataFrame()
df2['text'] = df['review']
df2['label'] = df['sentiment']

In [6]:
df2

,text,label
0,"Есть много причин, по которым 'Война и мир' Ль...",1
1,"Напишите 5 предложений. 1. ""Война и мир"" — это...",2
2,[Практикуйте «Другие люди»] Отзыв о «Войне и ...,1
3,Стремитесь к точному и объективному представле...,2
4,"В книге ""Война и мир"" Льва Толстого разворачив...",2
...,...,...
23995,"Вот и все, что нужно сделать: Во-первых, объяс...",1
23996,"Обязательно укажите, что именно в тексте/сюжет...",0
23997,"Кто, из героев вас заинтересовал и почему? Как...",1
23998,"Жду историй о том, как книга изменила ваше вос...",1


In [7]:
df2.to_csv('train2.csv',index = False)

In [8]:
# датасет для бинарной классификации
full_dataset_1 = load_dataset('csv', data_files=r'D:\data science\T1_hol\train2.csv')
# Фильтрация записей, в которых текстовые поля или метки не являются None
full_dataset_1 = full_dataset_1.filter(lambda example: example['text'] is not None and example['label'] is not None)

full_dataset_1

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/24000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 24000
    })
})

In [9]:
dataset = full_dataset_1['train'].train_test_split(test_size=0.2, seed=42)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 19200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 4800
    })
})

In [10]:
dataset['train'][0]

{'text': '(Читатель - представитель X-века, не читал классическую литературу)  ---  **Заголовок:** "Война и мир" Толстого: Почему он должен стать вашим первым шагом в классике XXI века  **Отзыв:**  В XXI веке, где мир сменяется в цифровых потоках, а информация доступна с одного касания, трудно представить, что книга, написанная более 150 лет назад, может предложить что-то действительно ценное. Однако "Война и мир" Льва Толстого - это именно тот роман, который стоит вашего времени и внимания, даже если вы никогда раньше не пробовали себя в чтении классической литературы.  **Почему "Война и мир"?**  1. **Мир, похожий на наш.** Да, события разворачиваются в начале XIX века, но человеческие чувства, стремления и страхи остаются неизменными. В романе вы найдете множество ситуаций, которые отражают сегодняшние проблемы: семейные конфликты, профессиональные разочарования, стремление к самореализации и поиск смысла жизни.  2. **Герои, как в соцсетях.** Представьте себе обширную сеть "друзей", 

In [11]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import torch

model_name = "blanchefort/rubert-base-cased-sentiment-rusentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3,ignore_mismatched_sizes=True)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [13]:
# токенизация текста
dataset = dataset.map(lambda e:
                      tokenizer(e['text'],
                                truncation = True,
                                max_length=512,
                                padding='max_length'), batched=True)

Map:   0%|          | 0/19200 [00:00<?, ? examples/s]

Map:   0%|          | 0/4800 [00:00<?, ? examples/s]

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 19200
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4800
    })
})

In [15]:
dataset = dataset.remove_columns('text')

In [16]:
dataset.set_format(type='torch')

In [17]:
dataset['train'][0]

{'label': tensor(1),
 'input_ids': tensor([  101,   120, 13687,  1023,   130, 11195,   216,   130,  4760,   128,
           802,  5390, 54532, 28977,   122,   130,   130,   130,   124,   124,
         64242,   156,   124,   124,   108, 20502,   322,  2498,   108, 34494,
           156,  2124,   988,  1967,  1989,  9191,  6661, 25860,   340, 66878,
         23181,  4760,   124,   124,  1486, 59778,   156,   124,   124,   436,
         23181,  9584,   128,  1177,  2498, 90480,   340, 39349, 19059,   381,
           128,   301,  6448, 31190,   336,  2260, 79216,   128,  4703,  4388,
           128,   825,  7628,   128, 30608,  1377,  4406,  1145,  2030,   128,
          1103, 12842,   825,   130,   822,  2089, 47795,   132,  3772,   108,
         20502,   322,  2498,   108, 30704, 34494,   130,   838,  1475,  1878,
          8261,   128,  1318,  1792,  5801,  1846,   322,  4797,   128,  1121,
          1007,   852,  2082,  2949,   802, 18614,  1326,   340, 36311, 24312,
         15398,   

In [18]:
outputs = model(dataset['train']['input_ids'][0].unsqueeze(0).to(device), dataset['train']['attention_mask'][0].unsqueeze(0).to(device), dataset['train']['label'][0].unsqueeze(0).to(device))
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.9719, -1.1868, -1.4693]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [19]:
from transformers import Trainer, TrainingArguments

# from datasets import load_metric
import evaluate
metric_f1 = evaluate.load("f1")
# Загрузка метрики вне функции
# metric_f1 = evaluate.load("accuracy")

# Параметры обучения
training_args = TrainingArguments(
    output_dir="cache",
    # learning_rate=3e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=15,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to = 'none'

)

# Функция для вычисления метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Для F1-score может потребоваться указать average параметр
    # 'micro', 'macro', 'weighted' или None в зависимости от задачи
    f1 = metric_f1.compute(predictions=predictions, references=labels, average="macro") # Или другой average
    return f1

trainer = Trainer(
    model=model,  # Модель для обучения
    args=training_args,  # Аргументы обучения
    train_dataset=dataset["train"],  # Набор данных для обучения
    eval_dataset=dataset["test"],  # Набор данных для валидации
    compute_metrics=compute_metrics,  # Метрики для вычисления
)

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.253500,0.200580,0.935419
2,0.117200,0.153786,0.954993
3,0.066500,0.193640,0.955181
4,0.043800,0.289744,0.955110
5,0.013900,0.279764,0.957655
6,0.007300,0.290337,0.955651
7,0.012700,0.355614,0.952305
8,0.006800,0.327636,0.957048
9,0.007800,0.319128,0.959798
10,0.002500,0.368897,0.956584


TrainOutput(global_step=9000, training_loss=0.03131592776067555, metrics={'train_runtime': 5163.7561, 'train_samples_per_second': 55.773, 'train_steps_per_second': 1.743, 'total_flos': 7.5776664305664e+16, 'train_loss': 0.03131592776067555, 'epoch': 15.0})

In [21]:
test = pd.read_csv(r'D:\data science\T1_hol\test.csv')

In [22]:
test

,index,review
0,690427,## Название: 'Война и мир': Грандиозное полотн...
1,253613,"Продукт не следует ни хвалить, ни критиковать,..."
2,379756,'Война и мир' Льва Толстого — это обширный ром...
3,696940,[Вопрос задан во время обсуждения другого вопр...
4,351121,Вот что я придумал: «Война и мир» Льва Толсто...
...,...,...
5995,796464,Лев Толстой. **Объективный отзыв о книге «Вой...
5996,148183,"Указанные темы, такие как «взаимоотношения, др..."
5997,239032,На английском языке Of all the literary works ...
5998,571112,"Тема: Разнотемпераментность персонажей, их хар..."


In [23]:

test2 = test.drop('index',axis = 1)

In [24]:
test = test.drop('review',axis =1)

In [25]:
test

,index
0,690427
1,253613
2,379756
3,696940
4,351121
...,...
5995,796464
5996,148183
5997,239032
5998,571112


In [26]:
t = pd.DataFrame()
t['Text'] = test2['review']
t

,Text
0,## Название: 'Война и мир': Грандиозное полотн...
1,"Продукт не следует ни хвалить, ни критиковать,..."
2,'Война и мир' Льва Толстого — это обширный ром...
3,[Вопрос задан во время обсуждения другого вопр...
4,Вот что я придумал: «Война и мир» Льва Толсто...
...,...
5995,Лев Толстой. **Объективный отзыв о книге «Вой...
5996,"Указанные темы, такие как «взаимоотношения, др..."
5997,На английском языке Of all the literary works ...
5998,"Тема: Разнотемпераментность персонажей, их хар..."


In [27]:
t.to_csv('test2.csv',index = False)

In [28]:
full_dataset_2 = load_dataset('csv', data_files=r'D:\data science\T1_hol\test2.csv')
# Фильтрация записей, в которых текстовые поля или метки не являются None
full_dataset_2 = full_dataset_2.filter(lambda example: example['Text'] is not None)

full_dataset_2

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/6000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Text'],
        num_rows: 6000
    })
})

In [29]:
pred_dataset = full_dataset_2['train']

pred_dataset = pred_dataset.map(lambda e: tokenizer(e['Text'],
                                truncation = True,
                                max_length=512,
                                padding='max_length'), batched=True)
# Remove the raw text column to avoid issues with the model
pred_dataset = pred_dataset.remove_columns("Text")

# Initialize the Trainer
trainer = Trainer(model=model)

# Make predictions
predictions = trainer.predict(pred_dataset)

# Get predicted class indices
predicted_class_indices = predictions.predictions.argmax(-1)


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [30]:
predicted_class_indices

array([0, 2, 2, ..., 1, 0, 0], dtype=int64)

In [31]:
test['sentiment'] = predicted_class_indices

In [32]:
test

,index,sentiment
0,690427,0
1,253613,2
2,379756,2
3,696940,0
4,351121,1
...,...,...
5995,796464,2
5996,148183,1
5997,239032,1
5998,571112,0


In [33]:
test.to_csv('otv.csv',index=False)